In [1]:
import torch
import torch.nn as nn
from src.models import SentimentModel

def test_model():
    print("Testing SentimentModel...")
    
    # Initialize model
    model = SentimentModel()
    print(f"Model initialized. Parameters: {sum(p.numel() for p in model.parameters()):,}")
    
    # Test data
    single_text = "This product is absolutely amazing!"
    batch_texts = [
        "Great quality, love it!",
        "Not worth the money, poor quality.",
        "It's okay, nothing special."
    ]
    
    print("\n=== FORWARD PASS TESTS ===")
    
    # Test 1: Single text forward pass
    print("\n1. Single text forward pass:")
    model.eval()
    with torch.no_grad():
        logits = model.forward(single_text)
        print(f"Input: '{single_text}'")
        print(f"Logits shape: {logits.shape}")
        print(f"Logits: {logits}")
        
    # Test 2: Batch forward pass
    print("\n2. Batch forward pass:")
    with torch.no_grad():
        batch_logits = model.forward(batch_texts)
        print(f"Batch size: {len(batch_texts)}")
        print(f"Logits shape: {batch_logits.shape}")
        print(f"Batch logits:\n{batch_logits}")
    
    # Test 3: Prediction methods
    print("\n3. Prediction methods:")
    predictions = model.predict(batch_texts)
    print(f"Probabilities:\n{predictions}")
    
    pred_with_conf = model.predict_with_confidence(batch_texts)
    print(f"Predictions: {pred_with_conf['predictions']}")
    print(f"Confidence scores: {pred_with_conf['confidence']}")
    
    print("\n=== BACKWARD PASS TEST ===")
    
    # Test 4: Backward pass (gradient computation)
    model.train()
    
    # Create dummy targets
    targets = torch.tensor([2, 0, 1])  # positive, negative, neutral
    
    # Forward pass
    logits = model.forward(batch_texts)
    
    # Compute loss
    criterion = nn.CrossEntropyLoss()
    loss = criterion(logits, targets)
    
    print(f"\n4. Backward pass:")
    print(f"Loss: {loss.item():.4f}")
    
    # Backward pass
    loss.backward()
    
    # Check if gradients were computed
    has_gradients = any(p.grad is not None for p in model.parameters())
    print(f"Gradients computed: {has_gradients}")
    
    # Print a few gradient norms to verify
    grad_norms = []
    for name, param in model.named_parameters():
        if param.grad is not None:
            grad_norm = param.grad.norm().item()
            grad_norms.append((name, grad_norm))
    
    print(f"\nSample gradient norms:")
    for name, norm in grad_norms[:5]:  # Show first 5
        print(f"  {name}: {norm:.6f}")
    
    print(f"\n✅ Model test completed successfully!")
    print(f"✅ Forward pass: Working")
    print(f"✅ Backward pass: Working") 
    print(f"✅ Ready for training!")

if __name__ == "__main__":
    test_model()

Testing SentimentModel...


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/48.0 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

Model initialized. Parameters: 66,365,187

=== FORWARD PASS TESTS ===

1. Single text forward pass:
Input: 'This product is absolutely amazing!'
Logits shape: torch.Size([1, 3])
Logits: tensor([[ 0.1999,  0.2188, -0.3922]])

2. Batch forward pass:
Batch size: 3
Logits shape: torch.Size([3, 3])
Batch logits:
tensor([[ 0.1701,  0.1471, -0.3121],
        [ 0.2056,  0.1642, -0.2232],
        [ 0.2624,  0.1894, -0.2855]])

3. Prediction methods:
Probabilities:
tensor([[0.3854, 0.3766, 0.2380],
        [0.3830, 0.3675, 0.2495],
        [0.3988, 0.3707, 0.2305]])
Predictions: [0 0 0]
Confidence scores: [0.3853998  0.38303182 0.3987667 ]

=== BACKWARD PASS TEST ===

4. Backward pass:
Loss: 1.1513
Gradients computed: True

Sample gradient norms:
  distilbert.embeddings.word_embeddings.weight: 0.649765
  distilbert.embeddings.position_embeddings.weight: 0.637908
  distilbert.embeddings.LayerNorm.weight: 0.048041
  distilbert.embeddings.LayerNorm.bias: 0.075643
  distilbert.transformer.layer.0.at

In [ ]:
from ml.src.training import train_sentiment_script

train_sentiment_script.train_sentiment_model()

ImportError: cannot import name 'SentimentLightningModule' from 'src.models' (/home/steff/workspace/csa/ml/src/models/__init__.py)